# Channels vs. Generators vs. Iterators  
Julia offers many way of dealing of large arrays while allocating as little of memory as possible. In this notebook, we attempt to makes a comparison of performance. The task we are interested in given to arrays, `x` and `y`, count number of times $$x_i^2+y_i^2>2y_i^2$$

We start by generating our two arrays. 

In [1]:
n_samples=100_000 # A hundered thousand samples
x=rand(1:10,n_samples)
y=rand(1:10,n_samples);

First we try to do things in specialized and efficient function

In [2]:
function fast_counter(x,y)
    n=0
    for i in  eachindex(x)
        n+=ifelse(x[i]^2+y[i]^2>2y[i]^2,true,false) 
    end
    n
end

fast_counter (generic function with 1 method)

We try it out

In [3]:
fast_counter(x,y)

45149

Now we benchmark it 

In [4]:
using BenchmarkTools
@btime fast_counter(x,y)

  118.653 μs (1 allocation: 16 bytes)


45149

Quite fast with almost not memory allocation. 

Now we try a solution that uses channels. First we write a function that generate a channel of specified size that is used to later to do the calculation. 

In [5]:
function channelGen(x,y,chn_size)
    chn=Channel(ctype=Bool,csize=chn_size) do c
        for i in eachindex(x)
            put!(c,x[i]^2+y[i]^2>2y[i]^2)
        end
    end
    chn
end

channelGen (generic function with 1 method)

The channel produces a `true` if the inequality is satisfied and `false` otherwise.  

Now we do the summation using our channel

In [6]:
function getsum(x,y, chn_size)
    c=channelGen(x,y, chn_size)
    sum(c)
end

getsum (generic function with 1 method)

We test

In [7]:
getsum(x,y, 10)

45149

Identical to earlier result. Now we benchmark  

In [8]:
@btime getsum(x,y, 10)

  27.469 ms (20029 allocations: 316.63 KiB)


45149

Waw! That is 270 times slower than the baseline. Lets see if channel size makes a difference

In [9]:
@btime getsum(x,y, 1000)

  6.972 ms (236 allocations: 11.53 KiB)


45149

Much better, only 70 times slower. Would a larger channels be better, lets try

In [10]:
@btime getsum(x,y, 10000)

  6.803 ms (60 allocations: 69.22 KiB)


45149

In [11]:
@btime getsum(x,y, 100_000)

  6.807 ms (41 allocations: 259.11 KiB)


45149

So larger Channel sizes are better, but only marginally so. A smaller channel is much worse due the blocking on the `put!` and `take!`

A more convenient style would be to employ a generator  

In [12]:
function getsum_gen(x,y) 
    sum(x[i]^2+y[i]^2>2y[i]^2 for i in eachindex(x))
end

getsum_gen (generic function with 1 method)

In [13]:
getsum_gen(x,y)

45149

In [14]:
@btime getsum_gen(x,y)

  125.431 μs (4 allocations: 96 bytes)


45149

This is just slightly slower than the baseline, but e code looks very clean. 

In contrast if we can try have a version this function with a standard array for the result before the summation. 

In [15]:
function getsum_arr(x,y) 
    r=[x[i]^2+y[i]^2>2y[i]^2 for i in eachindex(x)]
    sum(r)
end

getsum_arr (generic function with 1 method)

In [16]:
getsum_arr(x,y) 

45149

In [17]:
@btime getsum_arr(x,y)

  150.571 μs (6 allocations: 97.86 KiB)


45149

Slower with more memory allocation due to the generation of the intermediate `r` array.  

Now are ready to try out an Iterator implementation

In [18]:
struct BinGenrator{T}
    x::Vector{T}
    y::Vector{T}
    f::Function
end

Base.start(b::BinGenrator)=1
Base.next(b::BinGenrator,i)=(b.f(x[i],y[i]),i+1)
Base.done(b::BinGenrator,i)=i>length(b.x)

f(x,y)=x^2+y^2>2y^2
bg=BinGenrator(x,y,f)

sum(bg)

45149

In [19]:
@btime sum(bg)

  7.850 ms (297865 allocations: 4.55 MiB)


45149

This is even slower than the generator as doing too much allocation 

In [20]:
bg=BinGenrator(x,y,(x,y) -> x^2+y^2>2y^2)

sum(bg)

45149

In [21]:
@btime sum(bg)

  7.334 ms (297865 allocations: 4.55 MiB)


45149

...with an anonymous function it is faster! 

### In conclusion:
1. If you want top performance write specialized function like `fast_counter` 
2. Generators will be your second choice, a slight penalty in performance with elegant code
3. Storing the results in a intermediate array and then performing a final reduction on them is comparable to (2)
4. A suitably sized Channel can work even better than a custom iterator, but requires some experimentation
5. For trivial computation such shown here, the iterator interface might not be the best choice 